# 출석*Check*봇

### 모듈 import 및 그래프 폰트 설정


In [0]:
import random
# import numpy as np
import matplotlib.pyplot as plt

# # 그래프에서 한글표현을 위해 폰트를 설치합니다.
%config InlineBackend.figure_format = 'retina'
!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
# 기본 글꼴 변경
import matplotlib as mpl
mpl.font_manager._rebuild()
mpl.pyplot.rc('font', family='NanumBarunGothic')

### random 출석부 텍스트파일 생성

In [0]:
def create_sample():
    Checkbot_SampleData = open('checkbot_sample.txt', 'w', encoding='utf-8')
    name_list='''강수빈 이지환 김찬우 서샛별 고강문 고강우 권현준 정교윤 최태헌 
    황재익 조성준 이준기 이수현 이득기 문정식 이민진 김주찬 전은정 김미경 이진희
    김진성 임성학 이상균 장이욱 김도이 김지혜 진기성 소혜령 강경연 윤태연 장동희 
    최준열 김천실 박재형 신용승 박재은 소희철 남상이'''.split()

    date_list=list(range(20190901, 20190931))
    info = 'name'
    for date in date_list:
        info += ' '+str(date)

    Checkbot_SampleData.write(info+'\n')

    for name in name_list:
        s = name
        for _ in range(len(date_list)):
            s+=' '+random.choice(['y', 'n'])
        Checkbot_SampleData.write(s+'\n')

    Checkbot_SampleData.close()

### 텍스트파일 출력

In [0]:
def print_sample():
    with open('checkbot_sample.txt', 'r', encoding='utf-8') as f:
        data = f.read()
        print(data)


### 텍스트파일을 읽어서 자료구조에 저장

In [0]:
def file2dict(file):

    firstLine = file.readline() #string 리턴 해준다
    students = file.readlines() #원소는 텍스트한줄(string)인 list 를 리턴 해준다
    dateList = (firstLine.strip().split(" ")) #strip메소드는 양끝 개행문자/공백문자 제거

    resultDict = {}

    for student in students:

        #여기서 student는 텍스트 한줄(string)

        studentDict = {}
        studentInfo = student.strip().split()
        studentName = studentInfo[0]

        for idx in range(1, len(dateList)):
            studentDict[dateList[idx]] = studentInfo[idx]

        resultDict[studentName] = studentDict

    return resultDict


### 텍스트 파일 열어주는 함수

In [0]:
def file_open(file_path):
    try:
        f = open(file_path, "r", encoding="utf-8")
    except:
        print("파일이 없습니다.\n 샘플을 만들어서 파일을 열겠습니다.")
        create_sample()
        f = open("checkbot_sample.txt", "r", encoding='utf-8')
    return f

### 학생 이름과 날짜를 입력받고 반환하는 함수

In [0]:
def input_name_date(resultDict):
    print()
    print(list(resultDict.keys())) #이름 리스트 출력
    name_loop=True

    while name_loop:
        name = input("\n[checkbot] 학생의 이름을 입력하세요\n[관리자] => ")
        if name not in resultDict:
            print("[checkbot] 등록되지 않은 학생입니다. 다시 한번 확인 후에 입력해주세요\n")
        else:
            name_loop=False

    date_loop=True
    while date_loop:
        date = input("[checkbot] 날짜를 입력하세요[ex)20190901]\n[관리자] => ")
        if date not in resultDict[name]:
            print('[checkbot] 등록되지 않은 날짜입니다. [20190901~20190930]중 하나의 날짜를 8자리형식으로 다시 입력해주세요')
        else:
            date_loop=False
    return name,date

### 출석여부 확인해서 출력해주는 함수


In [0]:
def print_attedance(resultDict, name, date):
    if resultDict[name][date] == 'y':
        print("[checkbot] {}학생은 {}에 출석했습니다".format(name, date))
    else:
        print("[checkbot] 학생은 {}에 출석하지 않았습니다".format(name, date))


### 출석률 계산해서 반환해주는 함수

In [0]:
def attendanceRate(resultDict,name):

    attendanceCnt = 0

    for value in resultDict[name].values():
        if(value == 'y'):
            attendanceCnt += 1

    attendanceRate = attendanceCnt / len(resultDict[name]) * 100
    print('[checkbot] {}학생의 출석률은 {:.1f}%입니다'.format(name, attendanceRate))

### 그래프 출력해주는 함수

In [0]:
def make_graph(resultDict, name):
    if name not in resultDict:
        print("[checkbot] 등록되지 않은 학생입니다")
        return

    print('[checkbot] 그래프를 출력합니다.')


    x_list = []
    ynList = []

    for i, (n, attend_dict) in enumerate(resultDict.items()):
        total_attend = 0
        for yn in attend_dict.values():
            if yn == 'y':
                total_attend += 1

        ## 나머지 회원 이름은 비공개
        if name != n:
            n = 'ㅁㅁㅁ'
        else:
            idx = i

        x_list.append(n)
        ynList.append(total_attend)

    x = list(range(len(x_list)))
    fig = plt.figure(figsize=(22, 10))

    graph = plt.bar(range(len(x_list)), ynList)
    graph[idx].set_color('r')

    plt.title('출석')

    plt.xlabel('이름')
    plt.xticks(x, x_list)

    plt.ylabel('출석횟수')

    plt.show()



### 출석부 출력해주는 함수

In [0]:
def printDict(resultDict):
    print("=" * 100 + "출석부" + "="*100)
    for key, value in resultDict.items():
        print(key, value)
    print("="*203)

### 출석부 수정해주는 함수

In [0]:
def updateDict(resultDict):

    if adminCheck():
        print("[checkbot] 관리자 인증되었습니다.")
        name, date = input_name_date(resultDict)
        beforeUpdate = resultDict[name][date]
        print("\n[checkbot] {}학생의 {}출석기록은 {}입니다.\n".format(name, date, beforeUpdate))


        ynloop = True
        while(ynloop):
            yn = input("[checkbot] 수정 하시겠습니까?[y/n]\n[관리자] => ")

            if(yn=='y'):


                if resultDict[name][date]=='y':
                    resultDict[name][date] = 'n'
                else:
                    resultDict[name][date] = 'y'

                print("\n[checkbot] {}학생의 {}출석기록이 {}=>{}로 변경 되었습니다.\n".format(name,date,beforeUpdate,resultDict[name][date]))

                ynloop = False
                return

            elif(yn=='n'):
                ynloop = False
                return
            else:
                print("[checkbot] [y/n]입력오류입니다 다시 입력하세요")

    else:
        print("[checkbot] 비밀번호 오류입니다.")
        return

### 가능한 채팅 출력

In [0]:
def print_help():
    print('\n')
    print("="*60)
    print("어떤작업을 하실건가요? 번호를 입력해주세요")
    print("1. 학생의 출석정보 ")
    print("2. 출석부 수정하기")
    print("3. 출석부 출력")
    print("4. 출석률 50% 이하 학생들 확인하기")
    print("5. 출석률 50% 이하 학생 퇴원시키기")
    print("6. 특정 학생 퇴원시키기")
    print("7. checkbot 종료하기")
    print("=" *60)
    print('\n')

### 관리자 pw 인증 함수

In [0]:
def adminCheck():
    password = input("[checkbot] 관리자 비밀번호를 입력하세요\n[관리자] => ")
    if(password == "admin"):
        return True
    else:
        return False

### 네비게이터 

In [0]:
def navigator(resultDict):
    loop2 = True

    while (loop2):
        print_help()
        try:
            num = int(input("[checkbot] 번호를 입력하세요  \n[관리자] => "))

            if num == 1:
                # name과 date를 물어보고 알려줍니다.
                name, date = input_name_date(resultDict)
                print_attedance(resultDict, name, date)
                attendanceRate(resultDict, name)
                make_graph(resultDict, name)
            elif num == 2:
                updateDict(resultDict)

            elif num == 3:
                printDict(resultDict)

            elif num ==4:
                print("[checkbot] 아래의 학생들은 출석률 50% 미만입니다.")
                print(get_under50(resultDict))
            elif num == 5:
                if adminCheck():
                    delete_under50(resultDict)
                else:
                    print("[checkbot] 비밀번호 오류입니다")
            elif num == 6:
                    delete_name = input("[checkbot] 퇴원시킬 학생의 이름을 입력하세요\n[관리자] => ")
                    if(delete_name in resultDict):
                        if adminCheck():
                            del resultDict[delete_name]
                            print("[checkbot] {}학생을 성공적으로 삭제했습니다".format(delete_name))
                        else:
                            print("[checkbot] 비밀번호 오류입니다")
                    else:
                        print("[checkbot] 등록되지 않은 학생입니다")

            elif num == 7:
                print("\n[checkbot] checkbot을 이용해주셔서 감사합니다\n")
                loop2 = False
                return False
            else:
                print("[checkbot] 잘못된 번호를 입력하셨습니다 다시 입력해주세요")
        except:
            print("[checkbot] 잘못된 번호를 입력하셨습니다 다시 입력해주세요")


### 출석률 50% 미만의 학생들 반환

In [0]:
def get_under50(resultDict):
    under50s = []
    for name in list(resultDict.keys()):
        attendanceCnt = 0
        for value in list(resultDict[name].values()):
            if(value == 'y'):
                attendanceCnt += 1
        attendanceRate = attendanceCnt / len(resultDict[name]) * 100
        if attendanceRate < 50:
            under50s.append(name)

    return under50s

### 출석률 50% 미만의 학생들 출석부에서 삭제

In [0]:
def delete_under50(resultDict):

    under50s = get_under50(resultDict)
    for under50 in under50s:
        del resultDict[under50]
    print("[checkbot] 아래의 학생들이 출석부에서 삭제되었습니다. 출석부를 출력해서 확인할 수도 있습니다.")
    print(under50s)

### 챗봇 시나리오

In [0]:
def scenario(resultDict):
    loop = True

    while (loop):
        # 시작
        print("\n\n[checkbot] 안녕하세요 출석관리봇 checkbot입니다\n")
        loop = navigator(resultDict)


### main함수

In [0]:
def main():

    file = file_open("input.txt")
    resultDict = file2dict(file)
    printDict(resultDict)
    scenario(resultDict)
    file.close()

In [0]:
if __name__ == '__main__':
    main()